In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

In [3]:
messages = [
    SystemMessage(content="System"),
    HumanMessage(content="Human1"),
    AIMessage(content="AI1"),
    HumanMessage(content="Human2"),
    AIMessage(content="AI2"),
    HumanMessage(content="Human3"),
    AIMessage(content="AI3")
]

In [4]:
# 대화 많이 남기기.
trimmer = trim_messages(
    max_tokens = 1000,
    token_counter = llm
)
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [5]:
# 대화 적게 남기기.
trimmer = trim_messages(
    max_tokens = 30,
    token_counter = llm
)
trimmer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [6]:
# 과거 대화 남기고 최근 대화 지우기.
trimmer = trim_messages(
    max_tokens = 30,
    token_counter = llm,
    strategy = "first"
)
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={})]

In [7]:
# 최근 대화 남기고 과거 대화 지우기.
trimmer = trim_messages(
    max_tokens = 30,
    token_counter = llm,
    strategy = "last"
)
trimmer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [8]:
# 최근 대화 남기고 과거 대화 지우우는데 시스템 메세지는 남기기.
trimmer = trim_messages(
    max_tokens = 30,
    token_counter = llm,
    strategy = "last",
    include_system=True
)
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [9]:
# 최근 대화 남기고 과거 대화 지우우는데 시스템 메세지는 남기기.
# 그리고 Human 메세지부터 시작하기.
trimmer = trim_messages(
    max_tokens = 30,
    token_counter = llm,
    strategy = "last",
    include_system = True,
    start_on = "human"
)
trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [10]:
messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해."),
    HumanMessage(content="오늘은 피자를 먹어야지!"),
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇을 마실거야?"),
    
    HumanMessage(content="내일은 등산을 가야지!"),
    AIMessage(content="등산이라니, 정말 좋은 운동이야. 등산은 어디로 갈거야?"),
    
    HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?")
]

In [11]:
trimmer = trim_messages(
    max_tokens = 120,
    token_counter = llm,
    strategy = "last",
    include_system = True,
    start_on = "human"
)

In [12]:
trimmer.invoke(messages)

[SystemMessage(content='사용자의 질문에 2문장 이내로 짧게 대답해.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내일은 등산을 가야지!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='등산이라니, 정말 좋은 운동이야. 등산은 어디로 갈거야?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='주말에는 영화를 보러 갈 거야!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?', additional_kwargs={}, response_metadata={})]

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = trimmer | llm | parser

In [14]:
chain.invoke(messages + [
        HumanMessage(content="오늘 뭘 먹는다고 했지??")
    ]
)

'아쉽게도 기억이 나지 않아! 오늘 먹고 싶은 음식을 다시 생각해보는 건 어떨까?'

In [15]:
chain.invoke(messages + [
        HumanMessage(content="주말에는 뭘 한다고 했지??")
    ]
)

'주말에는 영화를 보러 간다고 했어! 어떤 영화인지 궁금해.'

In [16]:
import langchain
langchain.debug = True